<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a LlamaIndex Application</h1>

LlamaIndex provides high-level APIs that enable users to build powerful applications in a few lines of code. However, it can be challenging to understand what is going on under the hood and to pinpoint the cause of issues. Phoenix makes your LLM applications *observable* by visualizing the underlying structure of each call to your query engine and surfacing problematic `spans`` of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Build a simple query engine using LlamaIndex that uses retrieval-augmented generation to answer questions over the Arize documentation,
- Record trace data in [OpenInference tracing](https://github.com/Arize-ai/open-inference-spec/blob/main/trace/spec/traces.md) format using the global `arize_phoenix` handler
- Inspect the traces and spans of your application to identify sources of latency and cost,
- Export your trace data as a pandas dataframe and run an [LLM Evals](https://docs.arize.com/phoenix/concepts/llm-evals) to measure the precision@k of the query engine's retrieval step.

ℹ️ This notebook requires an OpenAI API key.

## 1. Install Dependencies and Import Libraries

Install Phoenix, LlamaIndex, and OpenAI.

In [ ]:
!pip install - qq "arize-phoenix[experimental]" gcsfs langchain "llama-index>0.8.36" openai


Import libraries.

In [ ]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import openai
import pandas as pd
import phoenix as px
from gcsfs import GCSFileSystem
from llama_index import ServiceContext, StorageContext, load_index_from_storage, set_global_handler
from llama_index.embeddings import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore
from llama_index.llms import OpenAI
from phoenix.experimental.evals import (
    OpenAIModel,
    compute_precisions_at_k,
    run_relevance_eval,
)
from tqdm import tqdm

pd.set_option("display.max_colwidth", 1000)


## 2. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the `OpenInferenceTraceCallbackHandler`. Phoenix supports LlamaIndex's [one-click observability](https://gpt-index.readthedocs.io/en/latest/end_to_end_tutorials/one_click_observability.html) which will automatically instrument your LlamaIndex application! You can consult our [integration guide](https://docs.arize.com/phoenix/integrations/llamaindex) for a more detailed explanation of how to instrument your LlamaIndex application.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run the LlamaIndex application).

In [ ]:
session = px.launch_app()


## 3. Configure Your OpenAI API Key

Set your OpenAI API key if it is not already set as an environment variable.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key


## 4. Build Your LlamaIndex Application

This example uses a `RetrieverQueryEngine` over a pre-built index of the Arize documentation, but you can use whatever LlamaIndex application you like.

Download our pre-built index of the Arize docs from cloud storage and instantiate your storage context.

In [ ]:
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),  # prevents unauthorized request to GCS
)


Enable Phoenix tracing within LlamaIndex by setting `arize_phoenix` as the global handler. This will mount Phoenix's [OpenInferenceTraceCallback](https://docs.arize.com/phoenix/integrations/llamaindex) as the global handler. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [ ]:
set_global_handler("arize_phoenix")


We are now ready to instantiate our query engine that will perform retrieval-augmented generation (RAG). Query engine is a generic interface in LlamaIndex that allows you to ask question over your data. A query engine takes in a natural language query, and returns a rich response. It is built on top of Retrievers. You can compose multiple query engines to achieve more advanced capability  

In [ ]:
service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
)
index = load_index_from_storage(
    storage_context,
    service_context=service_context,
)
query_engine = index.as_query_engine()


## 5. Run Your Query Engine and View Your Traces in Phoenix

We've compiled a list of commonly asked questions about Arize. Let's download the sample queries and take a look.

In [ ]:
queries_url = "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(queries_url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries[:10]


Let's run the first 10 queries and view the traces in Phoenix.

In [ ]:
for query in tqdm(queries[:30]):
    query_engine.query(query)


And just for fun, ask your own question!

In [ ]:
response = query_engine.query(
    "What is Arize and how can it help me as an AI Engineer?")
print(response)


Check the Phoenix UI as your queries run. Your traces should appear in real time.

Open the Phoenix UI with the link below if you haven't already and click through the queries to better understand how the query engine is performing. For each trace you will see a break

Phoenix can be used to understand and troubleshoot your by surfacing:
 - **Application latency** - highlighting slow invocations of LLMs, Retrievers, etc.
 - **Token Usage** - Displays the breakdown of token usage with LLMs to surface up your most expensive LLM calls
 - **Runtime Exceptions** - Critical runtime exceptions such as rate-limiting are captured as exception events.
 - **Retrieved Documents** - view all the documents retrieved during a retriever call and the score and order in which they were returned
 - **Embeddings** - view the embedding text used for retrieval and the underlying embedding model
LLM Parameters - view the parameters used when calling out to an LLM to debug things like temperature and the system prompts
 - **Prompt Templates** - Figure out what prompt template is used during the prompting step and what variables were used.
 - **Tool Descriptions** - view the description and function signature of the tools your LLM has been given access to
 - **LLM Function Calls** - if using OpenAI or other a model with function calls, you can view the function selection and function messages in the input messages to the LLM.

<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/RAG_trace_details.png" alt="Trace Details View on Phoenix" style="width:100%; height:auto;">

In [ ]:
print(f"🚀 Open the Phoenix UI if you haven't already: {session.url}")


## 6. Export and Evaluate Your Trace Data

You can export your trace data as a pandas dataframe for further analysis and evaluation.

In this case, we will export our `retriever` spans and evaluate each document retrieval so that we can compute an LLM-assisted precision@k. To learn more about the different span kinds, see the docs on [LLM Traces](https://docs.arize.com/phoenix/concepts/llm-traces)

In [ ]:
trace_df = px.active_session().get_spans_dataframe('span_kind == "RETRIEVER"')
trace_df


Evaluate your retrieval spans and surface problematic spans:

- Make LLM calls to classify each retrieved document as relevant or irrelevant to the corresponding query,
- Compute the precision@k for k = 1, 2 for each document,
- Sort your spans by precision@2 to surface up the most problematic spans.


In [ ]:
model = OpenAIModel(
    model_name="gpt-4",
    temperature=0.0,
)

trace_df["llm_assisted_relevance"] = run_relevance_eval(trace_df, model=model)
trace_df["llm_assisted_precision_at_k"] = (
    trace_df["llm_assisted_relevance"]
    .map(
        lambda predictions: [
            {"relevant": True, "irrelevant": False}.get(pred) for pred in predictions
        ]
    )
    .map(lambda x: compute_precisions_at_k(x) if x else float("nan"))
)
trace_df = trace_df.sort_values(
    by="llm_assisted_precision_at_k",
    key=lambda col: col.map(lambda x: x[-1] if isinstance(x, list) else 0.0),
    ascending=True,
)
trace_df[
    [
        "attributes.input.value",
        "attributes.retrieval.documents",
        "llm_assisted_relevance",
        "llm_assisted_precision_at_k",
    ]
]

Phoenix has build-in support for [LLM Evals](https://docs.arize.com/phoenix/concepts/llm-evals). With LLM Evals, you can quickly benchmark the performance of your LLM Application.

## 7. Final Thoughts

LLM Traces and the accompanying OpenInference Tracing specification is designed to be a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context such as retrieval from vector stores and the usage of external tools such as search engines or APIs. It lets you understand the inner workings of the individual steps your application takes wile also giving you visibility into how your system is running and performing as a whole. 

LLM Evals are designed for simple, fast and accurate LLM based evaluations. They let you quickly benchmark the performance of your LLM application and help you identify the problematic spans of execution.

For more details on Phoenix, LLM Tracing, and LLM Evals, checkout our [documentation](https://docs.arize.com/phoenix/).